In [1]:
import pandas as pd 
import numpy as np


path2 =  "/Users/tomelder/Downloads/vdjdb-2023-06-01/vdjdb_full.txt"
data = pd.read_csv(path2, sep='\t')
df = pd.DataFrame(data)

df = df[(df['vdjdb.score'] == 2) | (df['vdjdb.score'] == 3) | (df['vdjdb.score'] == 4)  | (df['vdjdb.score'] == 1) ]

relevant_columns = [
    'cdr3.alpha', 'v.alpha', 'j.alpha', 'cdr3.beta', 'v.beta', 'd.beta', 'j.beta',
    'species', 'mhc.a', 'mhc.b', 'antigen.gene','antigen.epitope', 'vdjdb.score', 'mhc.class'
]
# For the purpose of this example, we won't filter out lower-quality scores to avoid data imbalance,
# but in practice, you may want to consider this based on the dataset's size and distribution.
filtered_data = df[relevant_columns]
# Displaying the first few rows of the filtered dataset to verify the selection
filtered_data
# Checking for missing values in crucial columns
missing_values = filtered_data.isnull().sum()

# Examining the number of unique values in categorical columns for potential encoding strategies
unique_values = filtered_data.nunique()

# Removing rows with missing cdr3 sequences
df_cleaned = filtered_data.dropna(subset=['cdr3.alpha','cdr3.beta'])
df_cleaned.reset_index(drop=True, inplace=True)

missing_values_df_cleaned = df_cleaned.isnull().sum()
report = {
    "Missing Values": missing_values,
    "Missing Values df_cleaned": missing_values_df_cleaned
}

report

/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/2213001380.py:6: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path2, sep='\t')


{'Missing Values': cdr3.alpha         5559
 v.alpha            5585
 j.alpha            5740
 cdr3.beta           925
 v.beta              921
 d.beta             8374
 j.beta              996
 species               0
 mhc.a                 0
 mhc.b                 0
 antigen.gene         31
 antigen.epitope       0
 vdjdb.score           0
 mhc.class             0
 dtype: int64,
 'Missing Values df_cleaned': cdr3.alpha            0
 v.alpha               0
 j.alpha             122
 cdr3.beta             0
 v.beta                0
 d.beta             2656
 j.beta               74
 species               0
 mhc.a                 0
 mhc.b                 0
 antigen.gene         31
 antigen.epitope       0
 vdjdb.score           0
 mhc.class             0
 dtype: int64}

In [2]:
def filter_by_length_range(df, column_name):
    """
    Asks the user for length bounds and filters the DataFrame to include rows where the length of
    the specified column's sequence falls within the provided bounds.

    Parameters:
    df (DataFrame): The data frame to filter.
    column_name (str): The name of the sequence column to check (e.g., 'cdr3.alpha').

    Returns:
    DataFrame: A DataFrame filtered by the specified length range.
    """
    # Ask the user for length bounds
    lower_bound = int(input(f"Enter lower bound for {column_name} length: "))
    upper_bound = int(input(f"Enter upper bound for {column_name} length: "))
    
    # Calculate the sequence lengths
    df[column_name + '.length'] = df[column_name].apply(len)
    
    # Filter based on the length range
    return df[(df[column_name + '.length'] >= lower_bound) & (df[column_name + '.length'] <= upper_bound)]

def filter_by_species(df):
    """
    Asks the user for species to filter by and filters the DataFrame to include rows where the
    species column matches any of the species provided.

    Parameters:
    df (DataFrame): The data frame to filter.

    Returns:
    DataFrame: A DataFrame filtered by the specified species.
    """
    # Ask the user for species to filter by
    input_species = input("Enter the species to filter by (separated by commas): ")
    species_to_filter = [species.strip() for species in input_species.split(',')]
    
    return df[df['species'].isin(species_to_filter)]


def filter_by_minimum_score(df, column='vdjdb.score'):
    """
    Filters the DataFrame based on a minimum score inputted by the user for a specified column.
    
    Parameters:
    df (DataFrame): The data frame to filter.
    column (str): The name of the column to apply the filter on. Defaults to 'vdjdb.score'.
    
    Returns:
    DataFrame: A DataFrame filtered based on the user-specified minimum score.
    """
    # Prompting user for minimum score
    min_score = input(f"Enter the minimum score (inclusive) for {column}: ")
    
    # Validating user input
    try:
        min_score = int(min_score)
        if min_score < 0 or min_score > 3:
            print("Score out of range. Please enter a value between 0 and 3.")
            return df
    except ValueError:
        print("Invalid input. Please enter an integer value.")
        return df
    
    # Filtering the DataFrame
    filtered_df = df[df[column] >= min_score]
    
    return filtered_df

def filter_by_mhc_class(df, column='mhc.class'):
    """
    Filters the DataFrame based on a user-specified MHC class ('MHCI' or 'MHCII').

    Parameters:
    df (DataFrame): The data frame to filter.
    column (str): The name of the column to apply the filter on. Defaults to 'mhc.class'.

    Returns:
    DataFrame: A DataFrame filtered based on the user-specified MHC class.
    """
    # Prompting user for MHC class
    mhc_class = input(f"Enter the MHC class to keep ('MHCI' or 'MHCII'): ").strip()

    # Validating user input
    if mhc_class not in ['MHCI', 'MHCII']:
        print("Invalid input. Please enter 'MHCI' or 'MHCII'.")
        return df
    
    # Filtering the DataFrame
    filtered_df = df[df[column] == mhc_class]
    
    return filtered_df




# Assuming df is your DataFrame
df = df_cleaned
# Apply length range filters
df_filtered_alpha = filter_by_length_range(df, 'cdr3.alpha')
df_filtered_beta = filter_by_length_range(df, 'cdr3.beta')
df_filtered_epitope = filter_by_length_range(df, 'antigen.epitope')
df_filtered_min_score = filter_by_minimum_score(df)
# Intersect the filtered DataFrames to get only rows that meet all criteria
df_length_filtered = df_filtered_alpha.merge(df_filtered_beta).merge(df_filtered_epitope)

# Further filter by species
df_final_filtered = filter_by_species(df_length_filtered)

# df_final_filtered is now your preprocessed DataFrame



Enter lower bound for cdr3.alpha length: 10
Enter upper bound for cdr3.alpha length: 12


/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/163780384.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name + '.length'] = df[column_name].apply(len)


Enter lower bound for cdr3.beta length: 10
Enter upper bound for cdr3.beta length: 12


/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/163780384.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name + '.length'] = df[column_name].apply(len)


Enter lower bound for antigen.epitope length: 0
Enter upper bound for antigen.epitope length: 100


/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/163780384.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name + '.length'] = df[column_name].apply(len)


Enter the minimum score (inclusive) for vdjdb.score: 1
Enter the species to filter by (separated by commas): HomoSapiens


In [5]:
df_final_filtered

df_final_filtered.drop(['d.beta'], axis=1, inplace=True)

df_final_filtered.drop(['j.alpha'], axis=1, inplace=True)

print(df_final_filtered.isnull().sum())

cdr3.alpha                0
v.alpha                   0
cdr3.beta                 0
v.beta                    0
j.beta                    0
species                   0
mhc.a                     0
mhc.b                     0
antigen.gene              0
antigen.epitope           0
vdjdb.score               0
mhc.class                 0
cdr3.alpha.length         0
cdr3.beta.length          0
antigen.epitope.length    0
dtype: int64


/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/3232855618.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_filtered.drop(['j.alpha'], axis=1, inplace=True)


# CDR3 seq TCRdist

In [12]:
pip install tcrdist3==0.2.2

  Using cached numpy-1.22.4-cp39-cp39-macosx_10_15_x86_64.whl (17.7 MB)
  Using cached numpy-1.20.3-cp39-cp39-macosx_10_9_x86_64.whl (16.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
xarray 2023.8.0 requires numpy>=1.21, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 1.20.3 which is incompatible.
cartopy 0.22.0 requires numpy>=1.21, but you have numpy 1.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from tcrsampler.setup_db import install_all_next_gen
install_all_next_gen(dry_run = False)
from tcrdist.rep_funcs import _pws, _pw  
from tcrdist.repertoire import TCRrep

RUNNING: curl -o /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/wiraninha_sampler.zip https://www.dropbox.com/s/ily0td3tn1uc7bi/wiraninha_sampler.zip?dl=1 -L


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    66    0    66    0     0    122      0 --:--:-- --:--:-- --:--:--   122
100   320  100   320    0     0    277      0  0:00:01  0:00:01 --:--:--   277
100 6882k  100 6882k    0     0  2237k      0  0:00:03  0:00:03 --:--:-- 5306k
replace /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/wirasinha_mouse_alpha_g8a.tsv.sampler.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Archive:  /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/wiraninha_sampler.zip
RUNNING: curl -o /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/ravens_samplers.zip https://www.dropbox.com/s/bahxa6x86drq0n5/ravens_samplers.zip?dl=1 -L


100    64    0    64    0     0    198      0 --:--:-- --:--:-- --:--:--   198
100   320  100   320    0     0    269      0  0:00:01  0:00:01 --:--:--     0
100  313k  100  313k    0     0   158k      0  0:00:01  0:00:01 --:--:--  558k
replace /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/ravens_human_gamma_t.sampler.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Archive:  /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/ravens_samplers.zip
RUNNING: curl -o /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/olga_sampler.zip https://www.dropbox.com/s/qlsxvst8bn04l0n/olga_sampler.zip?dl=1 -L


...

100    92    0    92    0     0    264      0 --:--:-- --:--:-- --:--:--   264
100   320  100   320    0     0    385      0 --:--:-- --:--:-- --:--:--   385
100 11.3M  100 11.3M    0     0  3838k      0  0:00:03  0:00:03 --:--:-- 7236k
replace /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/emerson_human_beta_t_cmvneg.tsv.sampler.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Archive:  /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/emerson_human_beta_t_cmvneg.tsv.sampler.tsv.zip
RUNNING: curl -o /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/britanova_human_beta_t_cb.tsv.sampler.tsv.zip https://www.dropbox.com/s/87n5v2by80xhy1q/britanova_human_beta_t_cb.tsv.sampler.tsv.zip?dl=1 -L


100    90    0    90    0     0    263      0 --:--:-- --:--:-- --:--:--   263
100   320  100   320    0     0    364      0 --:--:-- --:--:-- --:--:--   364
100 28.3M  100 28.3M    0     0  5736k      0  0:00:05  0:00:05 --:--:-- 8742k


Archive:  /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/britanova_human_beta_t_cb.tsv.sampler.tsv.zip


replace /Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrsampler/db/britanova_human_beta_t_cb.tsv.sampler.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)
/Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [7]:
df_final_filtered.rename(columns={'cdr3.alpha': 'cdr3_a_aa', 'cdr3.beta': 'cdr3_b_aa', 'v.alpha':'v_a_gene','j.alpha': 'j_a_gene','v.beta': 'v_b_gene','j.beta': 'j_b_gene','antigen.epitope':'epitope'}, inplace=True)

print(np.shape(df_final_filtered))
def cdr3plot2(df_):
    # Assuming TCRrep is correctly set up to work with the provided dataframe
    tr_vdjdb = TCRrep(cell_df=df_, 
                      organism='human',
                      chains=['beta', 'alpha'],
                      deduplicate=False,
                      compute_distances=True )
    
    tcrdist_matrix = tr_vdjdb.pw_cdr3_b_aa
    return tcrdist_matrix

tcrdist_matrix =  cdr3plot2(df_final_filtered)
print(tcrdist_matrix)
print(np.shape(tcrdist_matrix))

/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/2049687273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_filtered.rename(columns={'cdr3.alpha': 'cdr3_a_aa', 'cdr3.beta': 'cdr3_b_aa', 'v.alpha':'v_a_gene','j.alpha': 'j_a_gene','v.beta': 'v_b_gene','j.beta': 'j_b_gene','antigen.epitope':'epitope'}, inplace=True)
/Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")
/Users/tomelder/opt/anaconda3/lib/python3.9/site-packages/tc

(182, 15)
[[ 0 24 24 ... 23 23 23]
 [24  0  0 ... 20 20 20]
 [24  0  0 ... 20 20 20]
 ...
 [23 20 20 ...  0  0  4]
 [23 20 20 ...  0  0  4]
 [23 20 20 ...  4  4  0]]
(182, 182)


### DBscan

In [9]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(metric='precomputed', eps=0.2, min_samples=2)  # Example: eps value here is hypothetical
clusters = dbscan.fit_predict(tcrdist_matrix)

# Adding cluster labels to your data (assuming you have a DataFrame 'df' with your TCR sequence data)
df_final_filtered['cluster'] = clusters

df_final_filtered

# Exclude outlier points and count unique clusters
n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)

print(f"Number of DBscan clusters: {n_clusters}")

print(f"Number of true clusters: {len(df_final_filtered['epitope'].unique())}")


Number of DBscan clusters: 11
Number of true clusters: 21


/var/folders/dg/1nsyrpm96v149hfhy4g6qc0h0000gn/T/ipykernel_53926/1066954043.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_filtered['cluster'] = clusters


### Random classifier for baseline


In [14]:
def random_clustering(df, n_clusters):
    """Assigns each row in the dataframe to a random cluster."""
    random_clusters = np.random.randint(0, n_clusters, len(df))
    df['random_cluster'] = random_clusters
    return df

def calculate_random_purity(df):
    purity_sum = 0
    for cluster in df['random_cluster'].unique():
        cluster_df = df[df['random_cluster'] == cluster]
        most_common_epitope = cluster_df['epitope'].value_counts().idxmax()
        purity_sum += cluster_df['epitope'].value_counts().max() / len(cluster_df)
    purity = purity_sum / len(df['random_cluster'].unique())
    return purity

def calculate_random_consistency(df, epitope_clusters):
    correct_assignments = 0
    for epitope, cluster in epitope_clusters.items():
        correct_assignments += len(df[(df['epitope'] == epitope) & (df['random_cluster'] == cluster)])
    consistency = correct_assignments / len(df)
    return consistency

def simulate_baseline(df, n_clusters, iterations=50):
    purity_scores = []
    consistency_scores = []
    def get_epitope_clusters(df):
        epitope_clusters = {}
        for epitope in df['epitope'].unique():
            epitope_df = df[df['epitope'] == epitope]
            most_common_cluster = epitope_df['cluster'].value_counts().idxmax()
            epitope_clusters[epitope] = most_common_cluster
        return epitope_clusters

    # Assuming df_final_filtered has 'cluster' column from DBSCAN and 'epitope' information
    epitope_clusters = get_epitope_clusters(df_final_filtered)

  # This should come from your actual clustering method

    for _ in range(iterations):
        df_random = random_clustering(df.copy(), n_clusters)
        purity = calculate_random_purity(df_random)
        consistency = calculate_random_consistency(df_random, epitope_clusters)
        
        purity_scores.append(purity)
        consistency_scores.append(consistency)
    
    return np.mean(purity_scores), np.mean(consistency_scores)

pur_base, consist_base = simulate_baseline(df_final_filtered, n_clusters)

In [15]:
def calculate_retention(df):
    # Retention is the fraction of TCR sequences assigned to any cluster
    assigned = df[df['cluster'] != -1]  # DBSCAN labels outliers as -1
    retention = len(assigned) / len(df)
    return retention

def calculate_purity(df):
    # Purity is defined for each cluster, then averaged across clusters
    purity_sum = 0
    for cluster in df['cluster'].unique():
        if cluster == -1:
            continue  # Skip noise points
        cluster_df = df[df['cluster'] == cluster]
        most_common_epitope = cluster_df['epitope'].value_counts().idxmax()
        purity_sum += cluster_df['epitope'].value_counts().max() / len(cluster_df)
    purity = purity_sum / (len(df['cluster'].unique()) - (1 if -1 in df['cluster'].unique() else 0))
    return purity

def calculate_consistency(df):
    # Consistency is calculated based on the assignment of TCR sequences to the "true" cluster for their epitope
    epitope_clusters = {}
    for epitope in df['epitope'].unique():
        epitope_df = df[df['epitope'] == epitope]
        most_common_cluster = epitope_df['cluster'].value_counts().idxmax()
        epitope_clusters[epitope] = most_common_cluster

    correct_assignments = 0
    for epitope, cluster in epitope_clusters.items():
        correct_assignments += len(df[(df['epitope'] == epitope) & (df['cluster'] == cluster)])

    consistency = correct_assignments / len(df[df['cluster'] != -1])
    return consistency



retention = calculate_retention(df_final_filtered)
purity = calculate_purity(df_final_filtered)
consistency = calculate_consistency(df_final_filtered)

print(f"Retention: {retention}")
print(f"Purity: {purity}")
print(f"Consistency: {consistency}")

print(f"Purity base: {pur_base}")
print(f"Consistency base: {consist_base}")

Retention: 0.7252747252747253
Purity: 0.9444444444444445
Consistency: 0.9318181818181818
Purity base: 0.37189435326019654
Consistency base: 0.0687912087912088
